In [1]:
import pandas as pd
from tqdm import tqdm 
import pytrec_eval
from collections import defaultdict
import pickle
import torch 
import torch.nn as nn
import os 
import numpy as np 
from transformers import AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding,AutoModel
from sentence_transformers import models, losses, LoggingHandler, SentenceTransformer, util, InputExample
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

def read_pickle(path):
	with open(path,'rb') as f:
		return pickle.load(f) 

## Load evaluation dataset (ESCO job normalization)

In [2]:
val_rqel = read_pickle('/mnt/data/nicky/JTG/jobbert-val-dict.pkl')
test_rqel = read_pickle('/mnt/data/nicky/JTG/jobbert-test-dict.pkl')

val_jts = read_pickle('/mnt/data/nicky/JTG/jobbert-val-jt.pkl')
test_jts = read_pickle('/mnt/data/nicky/JTG/jobbert-test-jt.pkl')

val_labels = read_pickle('/mnt/data/nicky/JTG/jobbert-val-labels.pkl')
test_labels = read_pickle('/mnt/data/nicky/JTG/jobbert-test-labels.pkl')

## Load pytrec to calculate Recall@k and MRR

In [3]:
val_evaluator = pytrec_eval.RelevanceEvaluator(
    val_rqel, {'recip_rank','recall'})
test_evaluator = pytrec_eval.RelevanceEvaluator(
    test_rqel, {'recip_rank','recall'})

## Evaluation Function

In [4]:
def eval_jobnorm(jts,labels,evaluator,model):
    encoded_query = model.encode(jts,normalize_embeddings = True,batch_size = 16,show_progress_bar = True)
    encoded_corpus = model.encode(labels,normalize_embeddings = True,batch_size = 16,show_progress_bar = True)
    sims_matrix = encoded_query @ encoded_corpus.T
    predict_q = dict()
    for i,q in enumerate(jts):
        sim = sims_matrix[i,:].astype(float)
        temp_q = dict()
        for j,p in enumerate(labels):
            temp_q[p] = sim[j]
        predict_q[q] = temp_q
    results = evaluator.evaluate(predict_q).items()
    
    mrr = np.mean([v['recip_rank'] for k,v in results])
    r_5 = np.mean([v['recall_5'] for k,v in results])
    r_10 = np.mean([v['recall_10'] for k,v in results])


    print('micro')
    print(f'R@5 = {r_5*100:.2f}, R@10 = {r_10*100:.2f}, mrr = {mrr*100:.2f}')

## BERT (w/o finetuning)

In [5]:
model_name = 'bert-base-uncased' 
model = SentenceTransformer(model_name,cache_folder = '/mnt/data/nicky/temp/.cache')

No sentence-transformers model found with name /mnt/data/nicky/temp/.cache/bert-base-uncased. Creating a new one with MEAN pooling.


None


In [6]:
eval_jobnorm(test_jts,test_labels,test_evaluator,model)

Batches:   0%|          | 0/967 [00:00<?, ?it/s]

Batches:   0%|          | 0/86 [00:00<?, ?it/s]

micro
R@5 = 26.23, R@10 = 32.10, mrr = 20.57


## JDAN (Ours)

In [7]:
model_name = 'napatnicky/JDAN-mycareersfuture.sg' 
# default (mean pooling)
model = SentenceTransformer(model_name,cache_folder = '/mnt/data/nicky/temp/.cache')

No sentence-transformers model found with name /mnt/data/nicky/temp/.cache/napatnicky_JDAN-mycareersfuture.sg. Creating a new one with MEAN pooling.


None


In [8]:
eval_jobnorm(test_jts,test_labels,test_evaluator,model)

Batches:   0%|          | 0/967 [00:00<?, ?it/s]

Batches:   0%|          | 0/86 [00:00<?, ?it/s]

micro
R@5 = 48.92, R@10 = 57.21, mrr = 39.12
